Neglate phase angle difference, charging and discharging rate

In [213]:
import grid_loader as gl
import pandas as pd
import numpy as np
import pandapower as pp

In [214]:
# Test case
n_case = 9

# Load test case
net = gl.load_test_case_grid(n_case)


In [215]:
# Define time periods
time_periods = range(0, 24)  # T

In [216]:

# Get load data
df_load_p = pd.read_csv("../Evaluation/Case%s/load_p.csv"%n_case).transpose()
df_load_q = pd.read_csv("../Evaluation/Case%s/load_q.csv"%n_case).transpose()

# Get renewable energy data
df_renewable = pd.read_csv("../Evaluation/Case%s/renewable.csv"%n_case).transpose()

# Get EV data
df_EV_spec = pd.read_csv("../Evaluation/Case%s/EV_spec.csv"%n_case)
df_EV_SOC = pd.read_csv("../Evaluation/Case%s/ev_soc.csv"%n_case).transpose()
df_EV_demand = pd.read_csv("../Evaluation/Case%s/ev_demand.csv"%n_case).transpose()


In [217]:
N_D = len(net.load.index)
# create storage as EV
for i in net.load.index:
    bus = net.load.loc[i, "bus"]
    n_car = int(net.load.loc[i, "p_mw"])  # number of EVs connected to the bus is assumed to be integer of nominal power of the loads
    max_e_mwh= df_EV_spec.loc[df_EV_spec["bus"] == bus, "max_e_mwh"].values[0]/n_car
    init_soc = df_EV_SOC.loc[:,0].iloc[i]
    pp.create_storage(
        net,
        bus=bus,
        p_mw=0,
        max_e_mwh= max_e_mwh,
        soc_percent=init_soc,
        min_e_mwh=df_EV_SOC.loc[:,0].iloc[i]*df_EV_spec.loc[df_EV_spec["bus"] == bus, "max_e_mwh"].values[0]/n_car,
        index=i,
        scaling=n_car,
        in_service=True,
        max_p_mw= min(0.05, max_e_mwh*(1-init_soc)), # maximum charging power = 50 kW
        min_p_mw= max(-0.05, -max_e_mwh*init_soc), # maximum discharging power = 50 kW
        max_q_mvar=0,
        min_q_mvar=0,
        controllable=True,
    )
    pp.create_load(net, bus=bus, p_mw=0, q_mvar=0, name="load_EV", index=i+N_D, in_service=True, type=None)

In [218]:
# Save results
df_cost = pd.DataFrame(index=time_periods)
df_gen_p = pd.DataFrame(index=time_periods, columns=net.gen.index)
df_gen_q = pd.DataFrame(index=time_periods, columns=net.gen.index)
df_charge = pd.DataFrame(index=time_periods, columns=net.storage.index)

for t in time_periods:
    if t == 0:
        net.storage.loc[:, "soc_percent"] = df_EV_SOC.loc[:, t].values
    for i in range(N_D):
        # Load data
        net.load.loc[i, "p_mw"] = df_load_p.loc[:,t].iloc[i]
        net.load.loc[i, "q_mvar"] = df_load_q.loc[:,t].iloc[i]

        # EV demand data
        net.load.loc[i+N_D, "p_mw"] = df_EV_demand.loc[:,t].iloc[i]

    # Renewable energy data
    net.gen.loc[:, "max_p_mw"]= df_renewable.loc[:, t].values


    # Run optimal power flow
    pp.runopp(net,verbose=False)
    df_cost.loc[t,"Cost $/hr"] = net.res_cost
    df_gen_p.loc[t] = net.res_gen.loc[:, "p_mw"]
    df_gen_q.loc[t] = net.res_gen.loc[:, "q_mvar"]
    df_charge.loc[t] = net.res_storage.loc[:, "p_mw"]

    # Update EV SOC
    for i in range(N_D):
        net.storage.loc[i, "soc_percent"] = (net.storage.loc[i, "soc_percent"]* net.storage.loc[i, "max_e_mwh"] + net.res_storage.loc[i, "p_mw"])/net.storage.loc[i, "max_e_mwh"] 
        net.storage.loc[i, "min_e_mwh"] = df_EV_SOC.loc[:,t].iloc[i]*net.storage.loc[i, "max_e_mwh"]
        if net.storage.loc[i, "soc_percent"] < df_EV_SOC.loc[:, t].iloc[i]:
            net.storage.loc[i, "min_p_mw"] = 0
        elif net.storage.loc[i, "soc_percent"] >= 1:
            net.storage.loc[i, "max_p_mw"] = 0
        else:
            net.storage.loc[i, "min_p_mw"] = -0.05
            net.storage.loc[i, "max_p_mw"] = 0.05


In [219]:
net.storage

,name,bus,p_mw,q_mvar,sn_mva,soc_percent,min_e_mwh,max_e_mwh,scaling,in_service,type,min_p_mw,max_p_mw,min_q_mvar,max_q_mvar,controllable
0,None,4,0.0,0.0,NaN,0.014184,0.044527,0.04500,90.0,True,None,0.0,0.000615,0.0,0.0,True
1,None,6,0.0,0.0,NaN,0.010264,0.044971,0.04500,100.0,True,None,0.0,0.001956,0.0,0.0,True
2,None,8,0.0,0.0,NaN,0.015165,0.045089,0.04561,125.0,True,None,0.0,0.000491,0.0,0.0,True


In [220]:
df_cost.sum()

Cost $/hr    142789.126237
dtype: float64